# Additional EDA

## To Do

- Train에서 공급유형이 `장기전세`, `공공분양`, `공공임대(5년)`인 example들의 `단지코드`가 차지하는 비율 살펴보기
- 하나의 단지코드내에 `임대건물구분` 값 존재 양상(상가/아파트/상가&아파트)에 따른 `등록차량수` 살펴보기
- `자격유형`별로 `임대보증금`과 `임대료`의 평균값들을 계산하여 소득수준과 관련한 판단 지표가 될 수 있는 지 확인하기.


## 살펴보면 좋을 것 같은 feature 관계

- 총세대수, 전용면적, 전용면적별세대수, 단지내주차면수 <==> 등록차량수
- 자하철, 버스 <==> 등록차량수

## Import Module

In [2]:
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
from matplotlib import pyplot as plt
import platform

if platform.system() == 'Window':
    font_name = font_manager.FontProperties(fname='c:/Windows/Fonts/malgun.ttf').get_name()
    rc('font', family=font_name)
else:
    rc('font', family='AppleChthic')

matplotlib.rcParams['axes.unicode_minus'] = False

## Data Load

In [3]:
DATA_ROOT = ''
DATA_ROOT = os.path.join(DATA_ROOT, "../../../competition_data/parking_data/")

TRAIN_ROOT = os.path.join(DATA_ROOT, "train.csv")
TEST_ROOT = os.path.join(DATA_ROOT, "test.csv")
AGE_GENDER_INFO_ROOT = os.path.join(DATA_ROOT, "age_gender_info.csv")

print(f"DATA_ROOT : {DATA_ROOT}")
print(f"TRAIN_ROOT : {TRAIN_ROOT}")
print(f"TEST_ROOT : {TEST_ROOT}")
print(F"SUBMISSION_ROOT : {AGE_GENDER_INFO_ROOT}")

DATA_ROOT : ../../../competition_data/parking_data/
TRAIN_ROOT : ../../../competition_data/parking_data/train.csv
TEST_ROOT : ../../../competition_data/parking_data/test.csv
SUBMISSION_ROOT : ../../../competition_data/parking_data/age_gender_info.csv


In [4]:
train = pd.read_csv(TRAIN_ROOT)
test = pd.read_csv(TEST_ROOT)
age_gender_info = pd.read_csv(AGE_GENDER_INFO_ROOT)

print("Data Loaded!")

Data Loaded!


## Preprocess (Version 2)

- 임대보증금, 임대료 -> `'-'` -> NULL, dtype을 float으로 변경
- NULL 값 처리
    - 임대보증금, 임대료, 지하철, 버스 -> fillna(0)
    - 자격유형 -> Case별 'A', 'C'로 채우기

### `'-'` -> NULL, dtype을 float으로 변경

In [5]:
columns = ['임대보증금', '임대료']

for col in columns:
    train.loc[train[col] == '-', col] = np.nan
    test.loc[test[col] == '-', col] = np.nan

    train[col] = train[col].astype(float)
    test[col] = test[col].astype(float)

### NULL 값 처리

#### 임대보증금, 임대료

In [6]:
train[['임대보증금', '임대료']] = train[['임대보증금', '임대료']].fillna(0)
test[['임대보증금', '임대료']] = test[['임대보증금', '임대료']].fillna(0)

#### 지하철, 버스

In [7]:
cols = ['도보 10분거리 내 지하철역 수(환승노선 수 반영)', '도보 10분거리 내 버스정류장 수']
train[cols] = train[cols].fillna(0)
test[cols] = test[cols].fillna(0)

#### 자격유형

In [8]:
test.loc[test.단지코드.isin(['C2411']) & test.자격유형.isnull(), '자격유형'] = 'A'
test.loc[test.단지코드.isin(['C2253']) & test.자격유형.isnull(), '자격유형'] = 'C'

#### 중복 example 제거

In [9]:
train = train.drop_duplicates()
test = test.drop_duplicates()

## Additional EDA

### Train에서 공급유형이 `장기전세`, `공공분양`, `공공임대(5년)`인 example들의 `단지코드`가 차지하는 비율 살펴보기

#### EDA 함수 정의

In [10]:
def describe_provide(data, value):

    print("\n==================================\n")

    # 해당 공급유형을 가지는 example 개수 출력
    print(f"공급유형이 '{value}'인 example의 수 : {len(data.loc[data.공급유형 == value, :])}")

    print("\n==================================\n")

    # 해당 공급유형을 가지는 단지 코드의 Unique 값 출력
    codes = data.loc[data.공급유형 == value, '단지코드'].unique().tolist()
    print(f"공급유형이 '{value}'인 단지코드 : {codes}")

    print("\n==================================\n")

    # 각 단지코드별 공급유형 양상 출력(value_counts)
    for code in codes:
        print(f"['{code}'의 공급유형 양상] : \n{data.loc[data.단지코드 == code, '공급유형'].value_counts()}\n")
    
    print("==================================\n")

    # 각 단지코드가 전체 단지코드에서 차지하는 비중 출력
    for code in codes:
        print("[전체 단지코드 중 '{}'의 비중] : {} / {} ({:.2f} %)\n".format(code, len(data.loc[data.단지코드 == code, :]), len(data), 100. * len(data.loc[data.단지코드 == code, :]) / len(data)))

    print("==================================\n")

#### 장기전세

In [11]:
train.loc[train.공급유형 == '장기전세', :]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
2043,C1397,370,아파트,서울특별시,장기전세,23.32,51,9.0,A,87444000.0,0.0,0.0,3.0,1590.0,1595.0
2044,C1397,370,아파트,서울특별시,장기전세,46.79,4,9.0,A,174888000.0,0.0,0.0,3.0,1590.0,1595.0
2045,C1397,370,아파트,서울특별시,장기전세,46.91,69,9.0,A,174888000.0,0.0,0.0,3.0,1590.0,1595.0
2046,C1397,370,아파트,서울특별시,장기전세,51.96,24,9.0,A,194562000.0,0.0,0.0,3.0,1590.0,1595.0
2047,C1397,370,아파트,서울특별시,장기전세,51.99,80,9.0,A,194562000.0,0.0,0.0,3.0,1590.0,1595.0
2048,C1397,370,아파트,서울특별시,장기전세,59.93,142,9.0,A,216423000.0,0.0,0.0,3.0,1590.0,1595.0
2100,C1039,790,아파트,서울특별시,장기전세,51.32,126,13.0,A,187694000.0,0.0,0.0,3.0,673.0,645.0
2101,C1039,790,아파트,서울특별시,장기전세,59.88,49,13.0,A,213863000.0,0.0,0.0,3.0,673.0,645.0
2102,C1039,790,아파트,서울특별시,장기전세,59.94,75,13.0,A,213863000.0,0.0,0.0,3.0,673.0,645.0


In [12]:
describe_provide(train, '장기전세')



공급유형이 '장기전세'인 example의 수 : 9


공급유형이 '장기전세'인 단지코드 : ['C1397', 'C1039']


['C1397'의 공급유형 양상] : 
장기전세    6
Name: 공급유형, dtype: int64

['C1039'의 공급유형 양상] : 
국민임대    6
장기전세    3
영구임대    2
Name: 공급유형, dtype: int64


[전체 단지코드 중 'C1397'의 비중] : 6 / 2632 (0.23 %)

[전체 단지코드 중 'C1039'의 비중] : 11 / 2632 (0.42 %)




#### 공공분양

In [13]:
train.loc[train.공급유형 == '공공분양', :]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
2331,C1350,1401,아파트,대전광역시,공공분양,74.94,317,2.0,D,0.0,0.0,0.0,6.0,1636.0,2315.0
2332,C1350,1401,아파트,대전광역시,공공분양,74.94,137,2.0,D,0.0,0.0,0.0,6.0,1636.0,2315.0
2333,C1350,1401,아파트,대전광역시,공공분양,74.94,22,2.0,D,0.0,0.0,0.0,6.0,1636.0,2315.0
2334,C1350,1401,아파트,대전광역시,공공분양,84.94,164,2.0,D,0.0,0.0,0.0,6.0,1636.0,2315.0
2335,C1350,1401,아파트,대전광역시,공공분양,84.94,19,2.0,D,0.0,0.0,0.0,6.0,1636.0,2315.0
2336,C1350,1401,아파트,대전광역시,공공분양,84.96,26,2.0,D,0.0,0.0,0.0,6.0,1636.0,2315.0
2337,C1350,1401,아파트,대전광역시,공공분양,84.97,26,2.0,D,0.0,0.0,0.0,6.0,1636.0,2315.0


In [14]:
describe_provide(train, '공공분양')



공급유형이 '공공분양'인 example의 수 : 7


공급유형이 '공공분양'인 단지코드 : ['C1350']


['C1350'의 공급유형 양상] : 
공공분양         7
공공임대(10년)    3
공공임대(분납)     3
Name: 공급유형, dtype: int64


[전체 단지코드 중 'C1350'의 비중] : 13 / 2632 (0.49 %)




#### 공공임대(5년)

In [15]:
train.loc[train.공급유형 == '공공임대(5년)', :]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
2580,C2470,636,아파트,대전광역시,공공임대(5년),39.90,96,1.0,A,23000000.0,340000.0,1.0,2.0,772.0,887.0
2581,C2470,636,아파트,대전광역시,공공임대(5년),46.81,264,1.0,A,26000000.0,380000.0,1.0,2.0,772.0,887.0
2582,C2470,636,아파트,대전광역시,공공임대(5년),59.95,46,1.0,A,32000000.0,480000.0,1.0,2.0,772.0,887.0


In [16]:
describe_provide(train, '공공임대(5년)')



공급유형이 '공공임대(5년)'인 example의 수 : 3


공급유형이 '공공임대(5년)'인 단지코드 : ['C2470']


['C2470'의 공급유형 양상] : 
공공임대(5년)    3
Name: 공급유형, dtype: int64


[전체 단지코드 중 'C2470'의 비중] : 3 / 2632 (0.11 %)




#### Result

- '장기전세', '공공분양', '공공임대(5년)' 인 example을 모두 Drop 하면, 
    - 전체 단지코드 중 9개 (0.34 %)의 단지 코드가 사라진다.
    - 전체 example 중 0.7% 가 사라진다.
    > 이 공급 유형들이 **Test 에 나타나지 않는다**는 점과 **전체 example 중 0.7 %**, 특히 이들을 **제거해도 전제 단지코드 중 0.34 % 의 단지코드만이 사라진다**는 점을 감안하면, 이들을 **제거하여 성능 평가를 해보는 것도 좋을 것** 같다.

### 하나의 단지코드내에 `임대건물구분` 값 존재 양상(상가/아파트/상가&아파트)에 따른 `등록차량수` 살펴보기

#### EDA 함수 정의

In [38]:
def car_per_building(data):
    codes = data.단지코드.unique().tolist()

    apartment_car = 0
    apartment_cnt = 0
    both_car = 0
    both_cnt = 0

    for code in tqdm(codes) :
        roi = data.loc[data.단지코드 == code, :]
        car = data.loc[data.단지코드 == code, '등록차량수'].unique().item()

        values = roi.임대건물구분.unique().tolist()

        if ('아파트' in values) & ('상가' in values):
            both_car += car
            both_cnt += 1
        elif '아파트' in values:
            apartment_car += car
            apartment_cnt += 1
    
    print("[각 단지 내에 임대건물구분이 '아파트'일 때 '평균 등록차량수'] : {:.2f}\n".format(apartment_car / apartment_cnt))
    print("[각 단지 내에 임대건물구분이 '상가&아파트'일 때 '평균 등록차량수'] : {:.2f}\n".format(both_car / both_cnt))

#### 각 단지 내 임대건물 유형 별 등록차량수 평균 분석

In [39]:
car_per_building(train)

  0%|          | 0/423 [00:00<?, ?it/s]

[각 단지 내에 임대건물구분이 '아파트'일 때 '평균 등록차량수'] : 608.13

[각 단지 내에 임대건물구분이 '상가&아파트'일 때 '평균 등록차량수'] : 149.36



#### Result

- 각 단지 내에 `임대건물구분`이 '아파트'일 때(608.13)가 '상가&아파트'일 때(149.36) 보다 등록차량수 가 더 높았다.
    > 즉, `주상복합`일 때 주차 수요가 더 높을 것이라는 예상은 옳지 못하다.
- '상가'만 존재하는 경우는 없었다.
    - 즉, '아파트'와 '상가&아파트' 의 양상만 보였다.
    - 전처리 Version 2 에서 처럼 feature를 늘리는 것 보다는....
    > '아파트'일 때와 '상가&아파트'일 때 2가지 경우로 LabelEncoding하는 것이 더 좋아보인다.

    > 단, '아파트'일 때의 주차 수요가 더 높았으므로 `'아파트'-> 1, '상가&아파트' -> 0` 으로 매핑해보자!

### `자격유형`별로 `임대보증금`과 `임대료`의 평균값들을 계산하여 소득수준과 관련한 판단 지표가 될 수 있는 지 확인하기.

#### EDA 함수 정의

In [106]:
def calc_mean_per_qualify(data):
    values = data.자격유형.unique().tolist()

    deposit = dict()
    rent = dict()
    deposit_rent = dict()

    for val in values:
        deposit[val] = data.loc[data.자격유형 == val, '임대보증금'].mean()
        rent[val] = data.loc[data.자격유형 == val, '임대료'].mean()
        data_agg = data.loc[data.자격유형 == val, '임대보증금'] + data.loc[data.자격유형 == val, '임대료']
        deposit_rent[val] = data_agg.mean()
    
    deposit = sorted(deposit.items(), reverse=True, key = lambda item: item[1])
    rent = sorted(rent.items(), reverse=True, key = lambda item: item[1])
    deposit_rent = sorted(deposit_rent.items(), reverse=True, key = lambda item: item[1])

    for key, val in deposit:
        print("[자격유형 '{}'의 평균 (임대보증금)] : {:.2f}".format(key, val))

    print("\n=============================\n")

    for key, val in rent:
        print("[자격유형 '{}'의 (평균 임대료)] : {:.2f}".format(key, val))
    
    print("\n=============================\n")

    for key, val in deposit_rent:
        print("[자격유형 '{}'의 평균 (임대보증금+임대료)] : {:.2f}".format(key, val))
    
    return deposit, rent, deposit_rent





def car_per_qualify(data, criteria=None):
    if criteria == None:
        values = data.자격유형.unique().tolist()
        car = dict()

        for val in values:
            car[val] = data.loc[data.자격유형 == val, '등록차량수'].mean()
        
        car = sorted(car.items(), reverse=True, key=lambda item:item[1])

        for key, val in car:
            print("['{}' 자격유형의 평균 등록차량수] : {:.2f}".format(key, val))

        
    else :
        values = list()

        for idx in range(len(criteria)):
            values.append(criteria[idx][0])

        for val in values:
            print("['{}' 자격유형의 평균 등록차량수] : {:.2f}".format(val, data.loc[train.자격유형 == val, '등록차량수'].mean()))

In [107]:
deposit, rent, deposit_rent = calc_mean_per_qualify(train)

[자격유형 'E'의 평균 (임대보증금)] : 91535140.54
[자격유형 'K'의 평균 (임대보증금)] : 47874181.82
[자격유형 'O'의 평균 (임대보증금)] : 41120000.00
[자격유형 'J'의 평균 (임대보증금)] : 32255628.57
[자격유형 'N'의 평균 (임대보증금)] : 31958000.00
[자격유형 'L'의 평균 (임대보증금)] : 30499272.73
[자격유형 'A'의 평균 (임대보증금)] : 26242505.35
[자격유형 'H'의 평균 (임대보증금)] : 25804461.04
[자격유형 'B'의 평균 (임대보증금)] : 16764833.33
[자격유형 'M'의 평균 (임대보증금)] : 14616000.00
[자격유형 'G'의 평균 (임대보증금)] : 13229222.22
[자격유형 'I'의 평균 (임대보증금)] : 9677122.45
[자격유형 'C'의 평균 (임대보증금)] : 6883108.70
[자격유형 'F'의 평균 (임대보증금)] : 4156666.67
[자격유형 'D'의 평균 (임대보증금)] : 0.00


[자격유형 'G'의 (평균 임대료)] : 233505.56
[자격유형 'O'의 (평균 임대료)] : 233010.00
[자격유형 'H'의 (평균 임대료)] : 213940.06
[자격유형 'A'의 (평균 임대료)] : 211465.00
[자격유형 'N'의 (평균 임대료)] : 151807.24
[자격유형 'L'의 (평균 임대료)] : 143742.12
[자격유형 'J'의 (평균 임대료)] : 141160.67
[자격유형 'B'의 (평균 임대료)] : 137213.89
[자격유형 'C'의 (평균 임대료)] : 101108.48
[자격유형 'M'의 (평균 임대료)] : 82820.00
[자격유형 'I'의 (평균 임대료)] : 82552.24
[자격유형 'F'의 (평균 임대료)] : 71470.00
[자격유형 'E'의 (평균 임대료)] : 47670.81
[자격유형 'K'의 (평균 임대료)] : 24930

In [108]:
car_per_qualify(train)

['H' 자격유형의 평균 등록차량수] : 914.43
['I' 자격유형의 평균 등록차량수] : 777.35
['G' 자격유형의 평균 등록차량수] : 748.56
['N' 자격유형의 평균 등록차량수] : 683.86
['A' 자격유형의 평균 등록차량수] : 678.96
['E' 자격유형의 평균 등록차량수] : 666.14
['B' 자격유형의 평균 등록차량수] : 558.28
['M' 자격유형의 평균 등록차량수] : 416.00
['O' 자격유형의 평균 등록차량수] : 416.00
['F' 자격유형의 평균 등록차량수] : 359.00
['K' 자격유형의 평균 등록차량수] : 332.06
['J' 자격유형의 평균 등록차량수] : 323.40
['L' 자격유형의 평균 등록차량수] : 277.55
['D' 자격유형의 평균 등록차량수] : 218.21
['C' 자격유형의 평균 등록차량수] : 173.59


#### Result

- 완벽하지는 않지만, '자격유형별 등록차량수'가 '자격유형별 평균 임대료'와 비슷한 양상을 보이는 듯 하다.
- 반면, '자격유형별 평균 임대보증금'은 같은 양상을 보인다고 판단하기 힘들었다.
> **`'자격유형별 평균 임대료'`를 feature로 이용하여 성능평가를 해보는 것도 좋아보인다.**